In [1]:
from loaders.data_loader import DataLoader
from models.builder import create_memory_network
from models.config import MemoryNetworkConfig, load_recent_weights, save_model

In [2]:
N_EPOCHS = 30
BATCH_SIZE = 32

In [3]:
config = MemoryNetworkConfig()
config.vocab_size = 1500
config.embedding_size = 50
config.dropout_rate = 0.3
config.n_lstm_nodes = 64
config.story_max_length = 500
config.query_max_length = 500
config.answer_max_length = 500 # seq2seq models
config.n_decoder_lstm = 3
config.n_encoder_lstm = 3

# Load Data

In [4]:
d_name = "sample_conversation"
loader = DataLoader(d_name)
train_data, test_data = loader.get_vectorized_data(config)
train_stories, train_queries, train_answers = train_data
test_stories, test_queries, test_answers = test_data 

/usr/local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


# Train Model

In [5]:
memory_network = create_memory_network(config)
memory_network.load_weights(f"weights/{d_name}.h5")

In [6]:
history = memory_network.fit([train_stories, train_queries], 
                             train_answers,
                             BATCH_SIZE, 
                             N_EPOCHS,
                             validation_data=([test_stories, test_queries], test_answers),)

Epoch 1/30
41/41 [==============================] - 44s 895ms/step - loss: 1.6485 - accuracy: 0.0281 - val_loss: 1.9610 - val_accuracy: 0.0322
Epoch 2/30
41/41 [==============================] - 34s 839ms/step - loss: 1.5963 - accuracy: 0.0367 - val_loss: 1.9553 - val_accuracy: 0.0332
Epoch 3/30
41/41 [==============================] - 34s 819ms/step - loss: 1.5404 - accuracy: 0.0374 - val_loss: 2.0045 - val_accuracy: 0.0338
Epoch 4/30
41/41 [==============================] - 34s 826ms/step - loss: 1.5784 - accuracy: 0.0367 - val_loss: 1.9652 - val_accuracy: 0.0337
Epoch 5/30
41/41 [==============================] - 34s 842ms/step - loss: 1.5391 - accuracy: 0.0367 - val_loss: 2.0265 - val_accuracy: 0.0191
Epoch 6/30
41/41 [==============================] - 34s 821ms/step - loss: 1.5719 - accuracy: 0.0357 - val_loss: 1.9768 - val_accuracy: 0.0333
Epoch 7/30
41/41 [==============================] - 34s 828ms/step - loss: 1.5613 - accuracy: 0.0362 - val_loss: 2.0027 - val_accuracy: 0.0340

In [7]:
memory_network.save(f"weights/{d_name}.h5")
print("saved!")

saved!


In [8]:
loader.decode_sequence(train_stories[98])

'sos 301 moved permanently moved permanently the he not moved here eos'

In [9]:
loader.decode_sequence(train_queries[98])

'sos til that all interaction no in bengals version resentment developers matt s encyclopedia inc to uniform video same this are their to terms ryan current for into upload eos'

In [10]:
loader.decode_one_hot(train_answers[98])

"sos keep i've about this in a document rush that they interest just worth for a z information drive eagles out a football eos"

In [11]:
preds = memory_network.predict([train_stories[:10], train_queries[:10]])
for i in range(len(preds)):
    p = preds[i]
    e = train_answers[i]
    print(i, ":", loader.decode_one_hot(p)[:50], "--", loader.decode_one_hot(e)[:50])

0 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos i rob for and to the of my for a history anoth
1 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos is has you're in errors eos
2 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos it you the of it's in denver and it events lea
3 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos that 7 is to that the surprises are am sometim
4 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos but the 564045497 they which too point is had 
5 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos i pyramids that i thinking league oscar pictur
6 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos hour information united i everyone the makes t
7 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos this ' has be the names chargers to soldier wi
8 : wikimedia wikimedia wikimedia wikimedia wikimedia  -- sos give never videos a a go 7 explain a vehicles 
9 : wikimedia wikimedia wikimedia wiki

In [12]:
loader.decode_one_hot(preds[1])

'wikimedia wikimedia wikimedia wikimedia wikimedia upcoming upcoming joke joke joke google sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos sos the the the the the the the the the the the the the the the eos eos'